In [1]:
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,random_split
from torchvision import datasets, transforms

import pandas as pd
import numpy as np

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [3]:
data = pd.read_csv("dummyDataTopData0.csv",header = None)
x=data.iloc[1:,1:5].astype('float32')
y=data.iloc[1:,-1].astype('float32')
x.iloc[:, 2] = x.iloc[:, 2] / 100
x.iloc[:, 3] = x.iloc[:, 3] / 100
y = y/max(y.values)

In [4]:
y

1         0.066667
2         0.633333
3         0.233333
4         0.200000
5         0.000000
            ...   
99996     0.000000
99997     0.000000
99998     0.300000
99999     0.600000
100000    0.000000
Name: 5, Length: 100000, dtype: float32

In [5]:
class WaterLevelDataset(Dataset):
    def __init__(self, x, y):
        x= torch.tensor(x,dtype=torch.float32)
        y = torch.tensor(y,dtype=torch.float32)
        self.X = x
        self.Y = y

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.Y[idx]]

In [6]:
dataset = WaterLevelDataset(x.values,y.values)

In [7]:
len(dataset.X)

100000

In [8]:
dataset.X

tensor([[0.0000, 0.0000, 0.1900, 0.3600],
        [1.0000, 1.0000, 0.9200, 0.0900],
        [0.0000, 1.0000, 0.4100, 0.6300],
        ...,
        [1.0000, 0.0000, 0.0100, 0.2500],
        [1.0000, 0.0000, 0.6500, 0.0000],
        [0.0000, 1.0000, 0.2700, 0.7400]])

In [9]:
dataset.Y

tensor([0.0667, 0.6333, 0.2333,  ..., 0.3000, 0.6000, 0.0000])

In [10]:
train, test = random_split(dataset, [int(len(dataset.X)*0.8),int(len(dataset.X)*0.2)])

In [11]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

## Model 1
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [18]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=self.layer2(X)
        X = self.activation(X)
        return X

# Instantiate the model
model = SimpleNN()



In [19]:
loss_fn = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)

In [20]:
for epoch in range(100):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 0.14711996912956238
Epoch [2/100], Loss: 0.12315593659877777
Epoch [3/100], Loss: 0.08295320719480515
Epoch [4/100], Loss: 0.10785600543022156
Epoch [5/100], Loss: 0.06765039265155792
Epoch [6/100], Loss: 0.07361288368701935
Epoch [7/100], Loss: 0.11493562161922455
Epoch [8/100], Loss: 0.07703375816345215
Epoch [9/100], Loss: 0.08906828612089157
Epoch [10/100], Loss: 0.09077414870262146
Epoch [11/100], Loss: 0.056827761232852936
Epoch [12/100], Loss: 0.09339269995689392
Epoch [13/100], Loss: 0.10400228202342987
Epoch [14/100], Loss: 0.09270545095205307
Epoch [15/100], Loss: 0.07810308784246445
Epoch [16/100], Loss: 0.13597312569618225
Epoch [17/100], Loss: 0.07594163715839386
Epoch [18/100], Loss: 0.054844990372657776
Epoch [19/100], Loss: 0.07078243792057037
Epoch [20/100], Loss: 0.05449756979942322
Epoch [21/100], Loss: 0.09989428520202637
Epoch [22/100], Loss: 0.10872489213943481
Epoch [23/100], Loss: 0.08325102925300598
Epoch [24/100], Loss: 0.04138816520571709

In [23]:
model.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            outputs = model(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx])
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")


Predicted: 9.40, Actual: 27.00
Predicted: 16.84, Actual: 30.00
Predicted: 14.13, Actual: 16.00
Predicted: 14.72, Actual: 15.00
Predicted: 16.63, Actual: 0.00
Predicted: 13.83, Actual: 6.00
Predicted: 7.34, Actual: 0.00
Predicted: 5.70, Actual: 0.00
Predicted: 17.43, Actual: 3.00
Predicted: 8.52, Actual: 2.00
Predicted: 15.36, Actual: 24.00
Predicted: 13.29, Actual: 16.00
Predicted: 17.23, Actual: 22.00
Predicted: 9.86, Actual: 19.00
Predicted: 3.89, Actual: 0.00
Predicted: 12.50, Actual: 6.00
Average loss on the testing dataset: 9.7718
Predicted: 14.84, Actual: 28.00
Predicted: 13.12, Actual: 3.00
Predicted: 15.34, Actual: 24.00
Predicted: 5.94, Actual: 0.00
Predicted: 6.98, Actual: 0.00
Predicted: 11.86, Actual: 20.00
Predicted: 12.20, Actual: 0.00
Predicted: 9.54, Actual: 27.00
Predicted: 4.52, Actual: 0.00
Predicted: 14.70, Actual: 8.00
Predicted: 10.16, Actual: 26.00
Predicted: 14.25, Actual: 18.00
Predicted: 14.00, Actual: 6.00
Predicted: 4.05, Actual: 0.00
Predicted: 14.89, Actua

In [32]:
torch.save(model, "model/model1TopData0-2.h5")

## Model 2
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [16]:
class Model2NN(nn.Module):
    def __init__(self):
        super(Model2NN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=torch.relu(self.layer2(X))
        X=self.layer3(X)
        X = self.activation(X)
        return X

# Instantiate the model
model1 = Model2NN()



In [17]:
loss_fn = nn.MSELoss()

optimizer = optim.SGD(model1.parameters(), lr=0.001)

In [18]:
for epoch in range(100):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model1(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 15.921584
Epoch [2/100], Loss: 10.256514
Epoch [3/100], Loss: 11.950041
Epoch [4/100], Loss: 9.892781
Epoch [5/100], Loss: 8.472719
Epoch [6/100], Loss: 9.007360
Epoch [7/100], Loss: 9.741856
Epoch [8/100], Loss: 10.104340
Epoch [9/100], Loss: 11.044621
Epoch [10/100], Loss: 7.148381
Epoch [11/100], Loss: 10.513676
Epoch [12/100], Loss: 9.482937
Epoch [13/100], Loss: 12.253504
Epoch [14/100], Loss: 8.160003
Epoch [15/100], Loss: 13.207208
Epoch [16/100], Loss: 8.312646
Epoch [17/100], Loss: 4.460102
Epoch [18/100], Loss: 10.828707
Epoch [19/100], Loss: 7.867447
Epoch [20/100], Loss: 8.247023
Epoch [21/100], Loss: 5.766013
Epoch [22/100], Loss: 5.689771
Epoch [23/100], Loss: 10.862436
Epoch [24/100], Loss: 7.120658
Epoch [25/100], Loss: 8.908924
Epoch [26/100], Loss: 12.093197
Epoch [27/100], Loss: 5.919594
Epoch [28/100], Loss: 8.327025
Epoch [29/100], Loss: 9.928624
Epoch [30/100], Loss: 4.254498
Epoch [31/100], Loss: 5.566569
Epoch [32/100], Loss: 8.547028
Epoch 

In [20]:
model1.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            outputs = model1(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx])
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")

Predicted: 3.55, Actual: 0.00
Predicted: 4.59, Actual: 0.00
Predicted: 15.87, Actual: 18.00
Predicted: 15.60, Actual: 6.00
Predicted: 14.24, Actual: 24.00
Predicted: 9.54, Actual: 30.00
Predicted: 14.44, Actual: 6.00
Predicted: 8.23, Actual: 2.00
Predicted: 15.67, Actual: 22.00
Predicted: 0.45, Actual: 0.00
Predicted: 15.22, Actual: 14.00
Predicted: 14.45, Actual: 27.00
Predicted: 12.44, Actual: 18.00
Predicted: 14.58, Actual: 8.00
Predicted: 1.96, Actual: 0.00
Predicted: 15.71, Actual: 11.00
Average loss on the testing dataset: 7.3171
Predicted: 14.22, Actual: 20.00
Predicted: 0.39, Actual: 0.00
Predicted: 14.82, Actual: 26.00
Predicted: 0.31, Actual: 0.00
Predicted: 14.54, Actual: 7.00
Predicted: 15.69, Actual: 30.00
Predicted: 14.70, Actual: 27.00
Predicted: 15.94, Actual: 21.00
Predicted: 14.37, Actual: 19.00
Predicted: 15.89, Actual: 18.00
Predicted: 4.91, Actual: 0.00
Predicted: 0.39, Actual: 0.00
Predicted: 14.27, Actual: 13.00
Predicted: 0.61, Actual: 0.00
Predicted: 1.89, Actu

In [21]:
torch.save(model1, "model/model2TopData0.h5")